In [21]:
import fluxions as fl
import numpy as np
from numpy import isclose

# Implementation decisions

## Flexible Syntax

We aimed to support as much syntax as reasonably possible. Consider the function:

In [87]:
x = fl.Var('x')
y = fl.Var('y')

f = x * y

### Simple derivative using dictionary variable binding

In [37]:
f.diff({'x': 2, 'y': 3})

array([[3., 2.]])

### Passing an array to bind variables

In [42]:
f.diff(np.array([[1, 3], [2, 4]]))

array([[3., 1.],
       [4., 2.]])

### Fluxion objects are aware of their expected input

In [48]:
f.shape()

(2, 1)

### Binding a mixture of scalars and vectors

In [55]:
f.diff({'x': 3, 'y': np.array([1,2])})

array([[1., 3.],
       [2., 3.]])

### Default values for variables

In [86]:
x = fl.Var('x', 2)
y = fl.Var('y')

g = x + y

g.val({'y': 1})

array([[3.]])

### Pre-bind variables for immediate evaluation

Rather than instantiating fluxion variable objects, only to immediately evaluate them, you can just pass raw values straight into fluxion functions. The ouput is a FluxionResult object of the form 
```python
(val, diff)
```
In this case evaluation occurs at the same that time the computation graph is being built. The computation graph is not built and then evaluated

In [63]:
fl.sin(np.pi)

FluxionResult(1.2246467991473532e-16,-1.0)

### Mix pre-binding with input values

In [64]:
g = fl.sin(np.pi) * fl.Var('x')

In [70]:
g.diff(5)

array([[-5.]])

## Behind the scenes implementation - Numpy

Our computation graph is evaluated using a dict of variable bindings on the way down, and numpy arrays on the way up

In [54]:
x = fl.Var('x')
y = fl.Var('y')

f = x * y

### Binding vector variables

f.diff({'x': np.array([1, 2]), 'y': np.array([3,4])})

We note that each node in the the computation graph returns a tuple of its values and its derivative at that point.

For example in the code above, the intermediate function calls would return:

```python

# f = x * y

# Evaluating Var('x')
(array([[1.], [2.]]), array([[1., 0.], [1., 0.]]))

# Evaluating Var('y')
(array([[3.], [4.]]), array([[0., 1.], [0., 1.]]))

# Evaluating Mul(x, y)
(array([[3.], [8.]]), array([[3., 1.], [4., 2.]]))
```